In [1]:
import json
import os

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import xarray as xr

In [2]:
filedir = '/glade/work/abaker/jian/rrtmgp/'
pydir = '/glade/u/home/abaker/repos/PyCECT/'

In [3]:
save_filename = filedir + 'pycect_results/savefile_t1.nc'
ds = xr.open_dataset(save_filename)
sum_filename = filedir + 'ens/sum.rrtmgp_cpu.ts1.nc'
sum_ds = xr.open_dataset(sum_filename)

In [4]:
# from savefile
test_size = ds.dims['test_size']
ens_size = ds.dims['ens_size']
nvars = ds.dims['nvars']

# get var names
vars = ds['vars'].values

# get test scores and means
t_scores = ds['scores'].values
t_std_gm = ds['std_gm'].values
t_gm = ds['gm'].values

# get ens pc scores distribution (nvars = npcs)
# these are mean zero, so array just contains sigma)
ens_score_dist = ds['ens_sigma_scores'].values
# this is the  standardized global means (nvars by ens_size) => so dist mean = 0 for each var
ens_std_gm = ds['ens_std_gm'].values

In [5]:
# from sumfile
ens_gm = sum_ds['global_mean'].values
P = sum_ds['loadings_gm'].values

In [6]:
str_vars = []
for i in range(vars.size):
    str_vars.append(vars[i].decode('UTF-8'))

In [7]:
t_scores.shape

(261, 3)

In [8]:
name = 'CLOUD'
var_id = str_vars.index(name)
var_id

13

In [9]:
var_id = 256

In [10]:
print(str_vars[var_id])
print('Ensemble standardized global mean distribution')
print('99.5 percentile = ', np.percentile(ens_std_gm[var_id, :], 99.5))
print('mean = ', ens_std_gm[var_id, :].mean())
print('min = ', ens_std_gm[var_id, :].min())
print('max = ', ens_std_gm[var_id, :].max())

soa_a2DDF
Ensemble standardized global mean distribution
99.5 percentile =  1.9380032053586154
mean =  -2.0737189743158523e-13
min =  -3.0244463610816514
max =  2.712823190006004


In [11]:
print(str_vars[var_id])
print('Test standardized gm')
t_std_gm[var_id, :]

soa_a2DDF
Test standardized gm


array([ 0.08914149, -0.44323204, -1.89011598])

In [12]:
print('Ensemble ORIG global mean distribution')
print('mean = ', ens_gm[var_id, :].mean())
print('min = ', ens_gm[var_id, :].min())
print('max = ', ens_gm[var_id, :].max())

Ensemble ORIG global mean distribution
mean =  1.0224689042417041e-15
min =  1.021540269608125e-15
max =  1.0233018572083804e-15


In [13]:
print('Test ORIG gm')
t_gm[var_id, :]

Test ORIG gm


array([1.02249627e-15, 1.02233281e-15, 1.02188856e-15])

In [14]:
# list all the failing PCs from run 1
# we have nvars PCs, but we are only looking at a subset (usually 50)
npc = 50
run = 0
for pc_i in range(npc):
    sig = ens_score_dist[pc_i]
    score = t_scores[pc_i, run]
    if abs(score) > 2.0 * sig:
        print('PC ', pc_i + 1)
        print('2*sigma = ', 2.0 * sig)
        print('test run score = ', score)

PC  6
2*sigma =  7.633742430563471
test run score =  -9.784577213794682
PC  9
2*sigma =  4.8829027415844575
test run score =  -8.2470911089691
PC  11
2*sigma =  4.076377235183916
test run score =  -6.913832203694438
PC  12
2*sigma =  3.852787926200952
test run score =  -8.726343198832296
PC  14
2*sigma =  3.489280656669743
test run score =  8.599997690649525
PC  16
2*sigma =  3.245409506909792
test run score =  -5.310481299643518
PC  20
2*sigma =  2.6754446863886727
test run score =  6.090352013547006
PC  21
2*sigma =  2.636489939029982
test run score =  6.263745020652159
PC  23
2*sigma =  2.406223171389322
test run score =  -5.352936225536784
PC  25
2*sigma =  2.262938335449712
test run score =  2.4264128700839223
PC  27
2*sigma =  2.2006675326243
test run score =  -2.47304527185811
PC  28
2*sigma =  2.0786132315256225
test run score =  -5.223176395478287
PC  32
2*sigma =  1.8444609407030084
test run score =  6.402810324660048
PC  34
2*sigma =  1.7630887020405785
test run score =  -2.

In [15]:
# remember 0-index here compared to the pyCECT output
# PC 6: failed 3 runs  [1, 2, 3]
# PC 7: failed 1 runs  [2]
# PC 9: failed 3 runs  [1, 2, 3]
# PC 11: failed 3 runs  [1, 2, 3]
# PC 12: failed 3 runs  [1, 2, 3]

pc_id = 5  # same as 6 above

In [16]:
print('ensemble: sigma = ', ens_score_dist[pc_id])
print('ensemble: 2*sigma = ', 2 * ens_score_dist[pc_id])

ensemble: sigma =  3.8168712152817355
ensemble: 2*sigma =  7.633742430563471


In [17]:
t_scores[pc_id, :]

array([ -9.78457721,  -9.14846334, -11.26124663])

In [18]:
# which variables contribute most to this pc?
# each column of P corresponds to the variable weights of each PC, so PC1 weights are in the first column (remember 0-indexing)
P.shape
wts = abs(P[:, pc_id])
sort_wts = np.argsort(wts)[::-1]

In [18]:
# look at the 20 biggest weights
sort_wts[:20]

array([256,  78,  70, 182, 235, 212, 211,  73,  72, 102,  13,  79, 181,
       108, 217, 216,   3,  55,  75, 224])

In [19]:
wts[sort_wts][:20]

array([0.15535576, 0.1484808 , 0.14026985, 0.13960554, 0.13703905,
       0.13234275, 0.13228836, 0.12591671, 0.12547456, 0.12240678,
       0.12212465, 0.12194505, 0.11930251, 0.11723972, 0.11666592,
       0.11659785, 0.11625502, 0.11443161, 0.11215421, 0.11135863])

In [20]:
# which vars are these?
for i in sort_wts[:20]:
    print(str_vars[i], ": ", wts[i])

soa_a2DDF :  0.15535576082001196
WPTHVP_CLUBB :  0.14848080289165497
VV :  0.14026985274737724
bc_c1SFWET :  0.13960554137545306
pom_c1SFWET :  0.13703905028239252
num_a1_CMXF :  0.13234274611540506
num_a1_CLXF :  0.13228836164794358
WP2_ZT_CLUBB :  0.12591671052080944
WP2_CLUBB :  0.12547455850410802
num_a4 :  0.12240678117481475
CLOUD :  0.12212464767931022
WSUB :  0.12194505276787321
bc_a4_SRF :  0.11930250851012852
pom_a4 :  0.11723971701756816
num_a2_CMXF :  0.1166659160427699
num_a2_CLXF :  0.11659785132333875
ANSNOW :  0.11625502403697753
THLM_CLUBB :  0.1144316078092251
WPRCP_CLUBB :  0.11215421165837641
num_a4_SRF :  0.11135863358402734


In [21]:
i = "soa_a2DDF"
if i in str_vars:
    print('yes')
    print(str_vars.index(i))

yes
256


In [19]:
import random

In [20]:
def random_pick(num_pick, end):
    ar = range(0, end)
    rand_list = random.sample(ar, num_pick)
    return rand_listz

In [21]:
def get_pertlim_uf(rand_num):
    i = rand_num
    if i == 0:
        ptlim = 0
    else:
        j = 2 * int((i - 1) / 100) + 101
        k = (i - 1) % 100
        if i % 2 != 0:  # odd
            ll = j + int(k / 2) * 18
            ippt = str(ll).zfill(3)
            ptlim = "0." + ippt + "d-13"
        else:  # even
            ll = j + int((k - 1) / 2) * 18
            ippt = str(ll).zfill(3)
            ptlim = "-0." + ippt + "d-13"
    return ptlim

'0.119d-13'

In [141]:
i = 902
print("i = ", i)
j = 2 * int((i - 1) / 100) + 101
print("j = ", j)
k = (i - 1) % 100
print("k = ", k)
if i % 2 != 0:  # odd
    ll = j + int(k / 2) * 18
    ippt = str(ll).zfill(3)
    print(ippt)
    ptlim = "0." + ippt + "d-13"

else:  # even
    ll = j + int((k - 1) / 2) * 18
    ippt = str(ll).zfill(3)
    print(ippt)
    ptlim = "-0." + ippt + "d-13"

print(ptlim)

i =  902
j =  119
k =  1
119
-0.119d-13


In [66]:
18 * 5

90

In [145]:
i = 4
print("i = ", i)
if i == 0:
    ptlim = 0
elif i >= 1800:
    print("don't support >= 2000")
else:
    if i < 900:
        j = 2 * int((i - 1) / 100) + 101
    elif i < 1000:
        j = 2 * int((i - 1) / 100) + 100
    elif i < 1900:
        j = 2 * int((i - 1001) / 100) + 102
    print("j = ", j)
    k = (i - 1) % 100  # this is just last 2 digits of i-1
    print("k = ", k)
    if i % 2 != 0:  # odd
        ll = j + int(k / 2) * 18
        ippt = str(ll).zfill(3)
        ptlim = "0." + ippt + "d-13"
    else:  # even
        ll = j + int((k - 1) / 2) * 18
        ippt = str(ll).zfill(3)
        ptlim = "-0." + ippt + "d-13"

print(ippt)
print(ptlim)

i =  4
j =  101
k =  3
119
-0.119d-13


In [1]:
import operator as op


def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if op.countOf(unique_list, x) == 0:
            unique_list.append(x)
        else:
            print(x)
    # print list
    # for x in unique_list:
    #    print(x)
    return unique_list

In [19]:
def big_get_pertlim_uf(rand_num):
    i = rand_num
    if i == 0:
        ptlim = 0
    elif i > 2000:
        print("don't support sizes > 2000")
    elif i <= 1800:
        if i <= 900:  # [1-900]
            j = 2 * int((i - 1) / 100) + 101
        elif i <= 1000:  # [901 - 1000]
            j = 2 * int((i - 1) / 100) + 100
        elif i <= 1800:  # [1001-1800]
            j = 2 * int((i - 1001) / 100) + 102
        # print("j = ", j)
        k = (i - 1) % 100  # this is just last 2 digits of i-1
        # print("k = ", k)
        if i % 2 != 0:  # odd
            ll = j + int(k / 2) * 18
            ippt = str(ll).zfill(3)
            ptlim = "0." + ippt + "d-13"
        else:  # even
            ll = j + int((k - 1) / 2) * 18
            ippt = str(ll).zfill(3)
            ptlim = "-0." + ippt + "d-13"
    else:  # [1801 - 2000]
        if i <= 1900:  # [1801-1900]
            j = 1
        else:  # [1901-2000]
            j = 2
        k = (i - 1) % 100
        if i % 2 != 0:  # odd
            ll = j + int(k / 2) * 2
            ippt = str(ll).zfill(3)
            ptlim = "0." + ippt + "d-13"
        else:  # even
            ll = j + int((k - 1) / 2) * 2
            ippt = str(ll).zfill(3)
            ptlim = "-0." + ippt + "d-13"

    return ptlim

In [20]:
big_get_pertlim_uf(901)

'0.118d-13'

In [21]:
big_get_pertlim_uf(899)

'0.999d-13'

In [22]:
pert_list = []
for i in range(2001):
    # print(i)
    pert_list.append(big_get_pertlim_uf(i))

In [23]:
len(pert_list)

2001

In [24]:
a = unique(pert_list)
len(a)

2001